# COAWST Dashboard
The COAWST Forecast is the only coupled ocean, atmosphere, wave and sediment transport model. Here we investigate the gridded COAWST forecast data products from [USGS Woods Hole THREDDS server](https://gamone.whoi.edu/thredds) and visualizing the data using the [pyviz](pyviz.org) tools. 

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset('http://gamone.whoi.edu/thredds/dodsC/coawst_4/use/fmrc/coawst_4_use_best.ncd')

Find all the data variables that depend on time (and are not time `bounds`)

In [ ]:
rho_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and (not 'offset' in var) and ('xi_rho' in ds[var].dims[-1]):
            rho_vars.append(var)

Import the [pyviz](http://pyviz.org) tools we need

In [ ]:
from cartopy import crs as ccrs
import hvplot.xarray
import holoviews as hv
from geoviews import tile_sources as gvts
import panel as pn

Create widget for variable selection

In [ ]:
var_select = pn.widgets.Select(name='COAWST Variables:', options=rho_vars, 
                               value='Hwave')

Create widget for basemap selection

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)                                                    

The `plot` function below creates the `hvplot` panel layout object.  We specify a basemap, pick the `quadmesh` plot type for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the `time` dimension so that specific times are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [ ]:
crs = ccrs.PlateCarree()

In [ ]:
def plot(var=None, base_map=None):
    base_map = base_map or base_map_select.value
    var = var or var_select.value
    mesh = ds[var][-24:,:,:].hvplot.quadmesh(x='lon_rho', y='lat_rho', rasterize=True, title=var,
                                    width=600, height=400, crs=crs,
                                    groupby=list(ds[var].dims[:-2]), cmap='jet')
    mesh.redim.default(**{d: atemp[d].values.max() for d in ds[var].dims[:-2]})
    overlay = (base_map * mesh.opts(alpha=0.7)).opts(active_tools=['wheel_zoom', 'pan'])
    widgets = {dim: pn.widgets.Select for dim in ds[var].dims[:-2]}
    return pn.pane.HoloViews(overlay, widgets=widgets).layout

In [ ]:
def on_var_select(event):
    var = event.obj.value
    dashboard[-1] = plot(var=var)

In [ ]:
def on_base_map_select(event):
    base_map = event.obj.value
    dashboard[-1] = plot(base_map=base_map)

In [ ]:
var_select.param.watch(on_var_select, parameter_names=['value']);
base_map_select.param.watch(on_base_map_select, parameter_names=['value']);

In [ ]:
dashboard = pn.Column(var_select, base_map_select, plot(var_select.value))

We use `.servable()` below not only to display the panel object, but to makes the panel servable outside the notebook via:  `panel serve COAWST_Dashboard.ipynb`

In [ ]:
dashboard.servable()